In [1]:
import numpy as np
import json
import sys
import pickle
import time
import cvxopt as cvx
# from cvxopt import matrix, solvers

In [30]:
def read_data (datafile="../dataset/SVM/train.csv", entryno=1):
    lines = np.array([(line.rstrip('\n')).split(',') for line in open(datafile)], dtype=np.int64)
    lines = lines[ (lines[:,-1] == entryno) | (lines[:,-1] == entryno+1) ]
    np.random.seed(0) # Deterministically random
    np.random.shuffle(lines)
#     print (lines[:10])
#     lines = lines[:200,:]
    X = lines[:,:-1] / 255.0
    Y = lines[:,-1] * 2 - 3 # for having y = {-1, 1}
    
#     print (X.shape, X)
#     np.set_printoptions(threshold=np.nan)
#     print (Y.shape, Y)
#     print (lines.shape, lines)
#     print (np.max(X[0]), np.min(X[0]))

    return X, Y
    
X, Y = read_data('../dataset/SVM/train.csv')
testX, testY = read_data('../dataset/SVM/test.csv')
print (X.shape, Y.shape, testX.shape, testY.shape)

(4000, 784) (4000,) (2167, 784) (2167,)


In [3]:
# LibSVM
from libsvm.python.svmutil import *

prob = svm_problem(Y, X)

param = svm_parameter()
param.kernel_type = RBF
param.gamma = 0.05
param.C = 1

m=svm_train(prob, param)

correct = 0
svm_predict(Y, X, m)[0]
result = testY == svm_predict(testY, testX, m)[0]
np.sum(result)

Accuracy = 99.975% (3999/4000) (classification)
Accuracy = 99.5847% (2158/2167) (classification)


2158

In [4]:
# Finding the SVM alphas
X = np.matrix(X)
Y = np.matrix(Y).T

m, n = X.shape
print (X.shape, Y.shape, m, n)

(4000, 784) (4000, 1) 4000 784


In [5]:
X[0] * X[1].T
M = X * X.T
# for i in range (m):
#     for j in range (m):
#         M[i][j] = float (X[i] * X[j].T)
        
print (M)

[[ 76.74978854  46.94712803  17.29138024 ...  21.15249519  30.8811534
   29.73359477]
 [ 46.94712803  61.93600923  22.13290273 ...  28.19813918  32.19226451
   28.4963168 ]
 [ 17.29138024  22.13290273  49.81311803 ...  50.78248366  22.85188774
   31.20730488]
 ...
 [ 21.15249519  28.19813918  50.78248366 ... 101.24009227  41.31810842
   59.17339485]
 [ 30.8811534   32.19226451  22.85188774 ...  41.31810842  72.36329104
   44.5817301 ]
 [ 29.73359477  28.4963168   31.20730488 ...  59.17339485  44.5817301
   86.90568243]]


In [6]:
mQ = Y * Y.T
print (mQ, mQ.shape)
# mQ = np.dot (mQ, M)
# print (mQ, mQ.shape)
mQ = np.multiply (mQ, M)
print (mQ, mQ.shape)
# for i in range (0, m):
#     for j in range (0, m):
#         Q[i][j] = 

[[ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 [ 1  1  1 ... -1 -1 -1]
 ...
 [-1 -1 -1 ...  1  1  1]
 [-1 -1 -1 ...  1  1  1]
 [-1 -1 -1 ...  1  1  1]] (4000, 4000)
[[ 76.74978854  46.94712803  17.29138024 ... -21.15249519 -30.8811534
  -29.73359477]
 [ 46.94712803  61.93600923  22.13290273 ... -28.19813918 -32.19226451
  -28.4963168 ]
 [ 17.29138024  22.13290273  49.81311803 ... -50.78248366 -22.85188774
  -31.20730488]
 ...
 [-21.15249519 -28.19813918 -50.78248366 ... 101.24009227  41.31810842
   59.17339485]
 [-30.8811534  -32.19226451 -22.85188774 ...  41.31810842  72.36329104
   44.5817301 ]
 [-29.73359477 -28.4963168  -31.20730488 ...  59.17339485  44.5817301
   86.90568243]] (4000, 4000)


In [7]:
Q = mQ

p = np.matrix(-np.ones((m, 1)))

G = np.matrix(np.vstack( (-np.identity(m), np.identity(m)) ))
h = np.matrix(np.vstack( (np.zeros((m,1)), np.ones((m,1))) ))

A = Y.T
b = 0

print (Q.shape, p.shape, G.shape, h.shape, A.shape, b)
# print (Q, p, G, h, A, b)

(4000, 4000) (4000, 1) (8000, 4000) (8000, 1) (1, 4000) 0


In [8]:
Q = cvx.matrix(Q)
p = cvx.matrix(p)
G = cvx.matrix(G)
h = cvx.matrix(h)
A = cvx.matrix(A, (1, m), 'd')
print (A, np.array(A).shape)
b = cvx.matrix(b, (1,1), 'd')

sol = cvx.solvers.qp(P=Q, q=p, G=G, h=h, A=A, b=b)

[-1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00 -1.00e+00  1.00e+00  1.00e+00 ... ]
 (1, 4000)
     pcost       dcost       gap    pres   dres
 0: -4.1525e+02 -8.7651e+03  5e+04  3e+00  7e-13
 1: -2.4967e+02 -5.2583e+03  1e+04  5e-01  5e-13
 2: -1.3844e+02 -2.0226e+03  3e+03  1e-01  3e-13
 3: -8.5841e+01 -9.5140e+02  2e+03  6e-02  2e-13
 4: -5.2441e+01 -6.5583e+02  1e+03  3e-02  2e-13
 5: -2.7968e+01 -4.4408e+02  7e+02  2e-02  1e-13
 6: -1.2542e+01 -2.5777e+02  4e+02  7e-03  1e-13
 7: -6.8992e+00 -1.6402e+02  2e+02  3e-03  1e-13
 8: -5.1204e+00 -7.1678e+01  8e+01  6e-04  9e-14
 9: -9.2479e+00 -4.0131e+01  3e+01  5e-05  9e-14
10: -1.2240e+01 -3.1494e+01  2e+01  4e-16  1e-13
11: -1.5470e+01 -2.3412e+01  8e+00  7e-15  9e-14
12: -1.6847e+01 -2.0347e+01  4e+00  8e-15  1e-13
13: -1.7813e+01 -1.8851e+01  1e+00  2e-15  1e-13
14: -1.8234e+01 -1.8322e+01  9e-02  7e-15  1e-13
15: -1.8273e+01 -1.8275e+01  2e-03  8e-15  1e-13
16: -1.8274e+01 -1.8274e+01  3e-05  2e-16  1e-13
17: -1.8274e+01 -1.8274e+01  3e

In [14]:
alphas = np.array(sol['x'])
print (type(alphas), alphas.shape)
print (alphas)

<class 'numpy.ndarray'> (4000, 1)
[[1.27869070e-11]
 [1.16482957e-10]
 [2.41937099e-11]
 ...
 [3.51591460e-11]
 [2.40105910e-11]
 [6.07657798e-12]]


In [10]:
# alphas = alphas[:,0]
# print (alphas)

[1.27869070e-11 1.16482957e-10 2.41937099e-11 ... 3.51591460e-11
 2.40105910e-11 6.07657798e-12]


In [11]:
A = np.matrix ([[1,2,3],[4,5,6]])
B = np.matrix ([[2],[3]])
print (A, B)
C = np.multiply(A, B)
C = np.multiply(C, B)
print (C)

[[1 2 3]
 [4 5 6]] [[2]
 [3]]
[[ 4  8 12]
 [36 45 54]]


In [12]:
print (Y.shape, X.shape, alphas.shape)

(4000, 1) (4000, 784) (4000,)


In [15]:
W = np.multiply(X, Y)
W = np.multiply(W, alphas)
print (W.shape, W)

(4000, 784) [[-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 [-0. -0. -0. ... -0. -0. -0.]
 ...
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]


In [16]:
W = W.sum(axis=0)
print (W.shape, W)

(1, 784) [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  9.81847586e-14
   4.77832492e-12  5.77231641e-12  5.70119397e-12  5.62949326e-12
   5.77763782e-12  6.69235666e-12  9.67854209e-12  7.30741071e-02
   1.35965757e-01  3.59380856e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 

In [17]:
W = np.array(W)[0, :]
print (W.shape, W)

(784,) [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  9.81847586e-14
  4.77832492e-12  5.77231641e-12  5.70119397e-12  5.62949326e-12
  5.77763782e-12  6.69235666e-12  9.67854209e-12  7.30741071e-02
  1.35965757e-01  3.59380856e-03  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  

In [18]:
print (np.sum(W))

27.517177723474614


In [19]:
epsilon = 1e-5
for idx, alp in enumerate(alphas):
    if (alp - 0 > epsilon and 1 - alp > epsilon):
        print (alp)
        y = float (Y[idx])
        w = np.matrix(W)
        x = np.matrix(X[idx, :]).T
        b = float (y - w*x)
        print (y, w.shape, x.shape, b)
        break
print (b)

[0.07195556]
1.0 (1, 784) (784, 1) -1.2195601219869507
-1.2195601219869507


In [20]:
W = np.matrix(W).T
print (W.shape, b, X.shape)

(784, 1) -1.2195601219869507 (4000, 784)


In [26]:
pred = testX*W
pred = np.array(pred)[:,0]
print (pred.shape, pred)

(2167,) [-2.78074852 -0.17536125 -0.62420218 ...  6.50380761  2.22197956
  5.40811221]


In [27]:
values = np.sign(pred)
print (values)

[-1. -1. -1. ...  1.  1.  1.]


In [29]:
Y = np.array(testY)[:,0]
print (Y)
accuracy = np.sum(values == Y) / len(Y)
print (accuracy)

[-1 -1 -1 ...  1  1  1]
0.9783110290724504


In [24]:
epsilon = 1e-5
sv = []
for idx, alp in enumerate(alphas):
    if (alp - 0 > epsilon and 1 - alp > epsilon):
#         print (alp, idx)
        sv.append(alp)
print (len(sv))

157
